# Extract and restore the onset apex and offset frames directory of SMIC, CASMEII and SAMM dataset

In [1]:
import os
import glob as gb
import pandas as pd
from tqdm import tqdm
from Visualize import mkdir

/home/s439476@ucstaff.win.canberra.edu.au/anaconda3/envs/keira_py37_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def read_csv(path: str, dataset:str) -> tuple:
    sr = 0 if dataset == 'CASMEII' else 13
    data = pd.read_excel(path,dtype={"Subject": str},skiprows = sr)
#     print(data)
    return data

In [3]:
def normSAMMname(img_root,subject,fra_id):
    fralist = [0,0,0]
    fralist[0] = os.path.join(img_root,subject+'_'+str(fra_id)+'.jpg')
    fralist[1] = os.path.join(img_root,subject+'_0'+str(fra_id)+'.jpg')
    fralist[2] = os.path.join(img_root,subject+'_00'+str(fra_id)+'.jpg')
    for frame_name in fralist:
        frame = cv2.imread(frame_name)
        if frame is not None:
            return frame_name  

In [4]:
def save_states(dataset: str, root_path: str, xlsx_data: pd.DataFrame):
    for idx in tqdm(range(len(xlsx_data))):
        #get emotion state frames name, directory and save path
        subject, onset_id, apex_id, off_id, rootdir, filedir = LoadingData(xlsx_data, root_path, idx)

        if dataset == 'CASMEII':
            #get frame directory   
            onset_frame_name = rootdir + f"reg_img{onset_id}.jpg"
            apex_frame_name = rootdir + f"reg_img{apex_id}.jpg"        
            offset_frame_name = rootdir + f"reg_img{off_id}.jpg"
        elif dataset == 'SAMM':
            onset_frame_name = normSAMMname(rootdir,subject,onset_id)
            apex_frame_name = normSAMMname(rootdir,subject,apex_id)
            offset_frame_name = normSAMMname(rootdir,subject,off_id)
            
        frame_list = [onset_frame_name, apex_frame_name, offset_frame_name]
        #save frame directory on .txt files
        savefame(filedir, frame_list)

In [5]:
def savefame(filedir, frame_list):
    file = open(filedir,'w')
    for frame in frame_list:
        file.write("{}\n".format(frame))
    file.close()

In [6]:
#get emotion state frames name, directory and save path
def LoadingData(data_info: pd.DataFrame, root_path: str, idx: int):
    #get emotion state frames name
    subject = data_info.loc[idx, "Subject"]
    filename = data_info.loc[idx, 'Filename']
    if dataset == 'CASMEII':
        onset_id = data_info.loc[idx, 'OnsetFrame']
        apex_id = data_info.loc[idx, 'ApexFrame']
        off_id = data_info.loc[idx, 'OffsetFrame']
    else:
        onset_id = data_info.loc[idx, 'Onset Frame']
        apex_id = data_info.loc[idx, 'Apex Frame']
        off_id = data_info.loc[idx, 'Offset Frame']
    
    #get the directory of each frame
    rootdir = os.path.join(root_path, subject, filename)

    #create save path for each file/video
    file_save_dir = os.path.join(data_save_dir, subject)
    mkdir(file_save_dir)
    filedir = os.path.join(file_save_dir, filename +'.txt')

    return subject, onset_id, apex_id, off_id, rootdir, filedir        

In [18]:
def save_SMIC_states(root: str, save:str):
    sub_path = os.path.join(root, 'subjects.txt')
    sub_list = open(sub_path).read().split('\n')[:-1]
    for sub in sub_list:
        #create save path for each file/video
        file_save_dir = os.path.join(data_save_dir, sub)
        root_dir = os.path.join(root, sub)
        mkdir(file_save_dir)
        video_path = os.path.join(root, sub, 'videos.txt')
        video_list = open(video_path).read().split('\n')[:-1]
        for vid in video_list:
            filename = vid.split(',')[0]
            filedir = os.path.join(file_save_dir, filename +'.txt')
            video = os.path.join(root_dir, filename +'.txt')
            frame_list = open(video).read().split('\n')[:-1] # frames in video
            onset_frame_name = frame_list[0]
            apex_frame_name = frame_list[int(len(frame_list)/2)]
            offset_frame_name = frame_list[-1]
            frame_list = [onset_frame_name, apex_frame_name, offset_frame_name]
    
            #save frame directory on .txt files
            savefame(filedir, frame_list)

In [19]:
global save_dir, data_save_dir

xlsx_dic = {'SAMM':'../SAMM/Raw_Data/SAMM/SAMM_Micro_FACS_Codes_v2.xlsx',
            'CASMEII':'../CASME-II/CASME2-coding-20190701.xlsx'}
root_dic = {'SAMM':'../SAMM_Cropped',
           'CASMEII':'../CASME-II/Cropped',
           'SMIC':'../MEGC_data/SMIC'}
save_dir = 'Dataset_EmotSta'
dataset = 'SMIC'
smi_root = '../MEGC_data/SMIC'
data_save_dir = os.path.join(save_dir, dataset)
save_SMIC_states(root_dic[dataset], data_save_dir)

In [7]:
for dataset in ['CASMEII','SAMM']:
    print('dataset:', dataset)
    #create save directory
    data_save_dir = os.path.join(save_dir, dataset)
    print('data_save_dir:', data_save_dir)
    #get xlsx data
    xlsx_data = read_csv(xlsx_dic[dataset], dataset)
    print('xlsx_data:', xlsx_data.size)
    #save the emotion states of each video
    save_states(dataset, root_dic[dataset], xlsx_data)

dataset: CASMEII
data_save_dir: Dataset_EmotSta/CASMEII
xlsx_data: 2295


 87%|█████████████████████████████████▉     | 222/255 [00:00<00:00, 2216.68it/s]

rootdir: ../CASME-II/Cropped/01/EP02_01f
filedir: Dataset_EmotSta/CASMEII/01/EP02_01f.txt
rootdir: ../CASME-II/Cropped/01/EP03_02
filedir: Dataset_EmotSta/CASMEII/01/EP03_02.txt
rootdir: ../CASME-II/Cropped/01/EP04_02
filedir: Dataset_EmotSta/CASMEII/01/EP04_02.txt
rootdir: ../CASME-II/Cropped/01/EP04_03
filedir: Dataset_EmotSta/CASMEII/01/EP04_03.txt
rootdir: ../CASME-II/Cropped/01/EP04_04
filedir: Dataset_EmotSta/CASMEII/01/EP04_04.txt
rootdir: ../CASME-II/Cropped/01/EP19_01
filedir: Dataset_EmotSta/CASMEII/01/EP19_01.txt
rootdir: ../CASME-II/Cropped/01/EP19_03f
filedir: Dataset_EmotSta/CASMEII/01/EP19_03f.txt
rootdir: ../CASME-II/Cropped/01/EP19_05f
filedir: Dataset_EmotSta/CASMEII/01/EP19_05f.txt
rootdir: ../CASME-II/Cropped/01/EP19_06f
filedir: Dataset_EmotSta/CASMEII/01/EP19_06f.txt
rootdir: ../CASME-II/Cropped/02/EP01_11f
filedir: Dataset_EmotSta/CASMEII/02/EP01_11f.txt
rootdir: ../CASME-II/Cropped/02/EP02_04f
filedir: Dataset_EmotSta/CASMEII/02/EP02_04f.txt
rootdir: ../CASME-II

100%|███████████████████████████████████████| 255/255 [00:00<00:00, 2238.60it/s]


rootdir: ../CASME-II/Cropped/26/EP13_01
filedir: Dataset_EmotSta/CASMEII/26/EP13_01.txt
rootdir: ../CASME-II/Cropped/26/EP13_02
filedir: Dataset_EmotSta/CASMEII/26/EP13_02.txt
rootdir: ../CASME-II/Cropped/26/EP13_11
filedir: Dataset_EmotSta/CASMEII/26/EP13_11.txt
rootdir: ../CASME-II/Cropped/26/EP15_01
filedir: Dataset_EmotSta/CASMEII/26/EP15_01.txt
rootdir: ../CASME-II/Cropped/26/EP16_01
filedir: Dataset_EmotSta/CASMEII/26/EP16_01.txt
rootdir: ../CASME-II/Cropped/26/EP18_44
filedir: Dataset_EmotSta/CASMEII/26/EP18_44.txt
rootdir: ../CASME-II/Cropped/26/EP18_46
filedir: Dataset_EmotSta/CASMEII/26/EP18_46.txt
rootdir: ../CASME-II/Cropped/26/EP18_47
filedir: Dataset_EmotSta/CASMEII/26/EP18_47.txt
rootdir: ../CASME-II/Cropped/26/EP18_49
filedir: Dataset_EmotSta/CASMEII/26/EP18_49.txt
rootdir: ../CASME-II/Cropped/26/EP18_50
filedir: Dataset_EmotSta/CASMEII/26/EP18_50.txt
rootdir: ../CASME-II/Cropped/26/EP18_51
filedir: Dataset_EmotSta/CASMEII/26/EP18_51.txt
dataset: SAMM
data_save_dir: Dat

  0%|                                                   | 0/159 [00:00<?, ?it/s]


rootdir: ../SAMM_Cropped/006/006_1_2
filedir: Dataset_EmotSta/SAMM/006/006_1_2.txt


NameError: name 'cv2' is not defined